# Creating a Multivariate Linear Regression Pipeline to Predict House Sale Prices

This project works with housing data for the city of Ames, Iowa, United States from 2006 to 2010. You can read more about why the data was collected here https://doi.org/10.1080/10691898.2011.11889627. You can also read about the different columns in the data here https://s3.amazonaws.com/dq-content/307/data_description.txt. The pipeline
- cleans the data
- selects the most promising features, including both numerical and categorical features converted to dummy variables
- trains and tests a multivariate linear regression model on the selected features, cross-validating with sklearn's KFold function
- is written in such a way that it could be relatively easily adapted to another dataset.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [2]:
housing_data = pd.read_csv('AmesHousing.tsv', sep='\t')

In [3]:
def transform_features(data):
    train = housing_data.copy()
    
    ## Drop any column with more than some cutoff-percentage missing values
    thresh = .05
    train.dropna(thresh = len(train) * thresh, axis = 1, inplace = True)

    # Series object: column name -> number of missing values
    num_missing = train.isnull().sum()

    # Filter series to columns containing >5% missing values
    drop_missing_cols = num_missing[
        (num_missing > len(train) * thresh)].sort_values()

    # Drop those columns from the data frame. Note the use of the .index
    train = train.drop(drop_missing_cols.index, axis=1)

    # Drop any other columns we don't want in the model
    train = train.drop(['PID', 'Order'], axis=1)


    ### Transform features into proper format, i.e. numerical to categorical,
    ### scaling numerical, filling missing values, etc.

    ## For columns with fewer than 5% missing values, fill with mean
    # Get null counts for each column
    train_null_counts = train.isnull().sum()

    # Select columns with <5% missing values
    df_missing_values = train_null_counts[
        (train_null_counts < 0.05 * len(train))]
    df_missing_values = train[df_missing_values.index]


    ## Impute missing values
    # Compute column-wise missing value counts
    num_missing = train.select_dtypes(
        include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[
        (num_missing < len(train) * thresh) & (num_missing > 0)].sort_values()

    # Compute the most common value for each column
    replacement_values_dict = train[
        fixable_numeric_cols.index].mode().to_dict(orient='records')[0]

    # Use `pd.DataFrame.fillna()` to replace missing values.
    train = train.fillna(replacement_values_dict)


    ## Drop any text columns with >1 missing value
    # Create series object: column name -> number of missing values
    text_mv_counts = train.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

    # Filter series to columns containing *any* missing values
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]

    train = train.drop(drop_missing_cols_2.index, axis=1)


    ## Create new features by combining other features
    years_sold = train['Yr Sold'] - train['Year Built']
    neg_indices = set(years_sold.index[years_sold < 0].tolist())

    years_since_remod = train['Yr Sold'] - train['Year Remod/Add']
    neg_indices.update(
        years_since_remod.index[years_since_remod < 0].tolist())

    ## Create new columns
    train['Years Before Sale'] = years_sold
    train['Years Since Remod'] = years_since_remod

    ## Drop rows with negative values for both of these new features
    train = train.drop(neg_indices, axis=0)

    ## No longer need original year columns
    train = train.drop(["Year Built", "Year Remod/Add"], axis = 1)

    ## Drop any column that leaks info about the sale, such as Sale Year
    train = train.drop(
        ['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition'], axis=1)
    
    return train

In [4]:
def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(
        abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    # All categorical columns in the original dataset
    nominal_features = [
        "PID", "MS SubClass", "MS Zoning", "Street", "Alley", 
        "Land Contour", "Lot Config", "Neighborhood", "Condition 1", 
        "Condition 2", "Bldg Type", "House Style", "Roof Style", 
        "Roof Matl", "Exterior 1st", "Exterior 2nd", "Mas Vnr Type", 
        "Foundation", "Heating", "Central Air", "Garage Type", 
        "Misc Feature", "Sale Type", "Sale Condition"]
    
    # Reduce list to columns we still have
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    # Drop categorical columns with too many unique values; default 10
    uniqueness_counts = df[transform_cat_cols].apply(
        lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    # Select just the remaining text columns and convert to categorical
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    
    # Create dummy columns and add back to the dataframe!
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(
        include=['category']))], axis=1)
    
    return df

In [5]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = LinearRegression()
    
    # Holdout validation
    if k == 0:
        # Splits dataset into training and testing set
        train = df[:1460]
        test = df[1460:]

        # Define regression training set
        X = train[features]
        y = train['SalePrice']

        # Train model using selected features from training set
        reg = LinearRegression().fit(X, y)

        # Predict sale prices on testing set using features
        test_pr = reg.predict(test[features])

        # Calculate RMSE for test prediction set
        test_rmse = np.sqrt(mean_squared_error(test['SalePrice'], test_pr))

        return test_rmse
    
    # Simple cross validation
    elif k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        fold_one = df[:1460]
        fold_two = df[1460:]
        
        # Splits dataset into two folds
        fold_one = df[:1460]
        fold_two = df[1460:]

        # Define regression training set using first fold
        X = fold_one[features]
        y = fold_one['SalePrice']

        # Train model using selected features from training set
        reg = LinearRegression().fit(X, y)

        # Predict sale prices on fold_two using features
        fold_two_pred = reg.predict(fold_two[features])

        # Calculate RMSE for test prediction set
        rmse_one = np.sqrt(
            mean_squared_error(fold_two['SalePrice'], fold_two_pred))
        print('RMSE one: ', rmse_one)
        
        
        # Define regression training set using second fold
        X = fold_two[features]
        y = fold_two['SalePrice']

        # Train model using selected features from training set
        reg = LinearRegression().fit(X, y)

        # Predict sale prices on fold_one using features
        fold_one_pred = reg.predict(fold_one[features])

        # Calculate RMSE for test prediction set
        rmse_two = np.sqrt(
            mean_squared_error(fold_one['SalePrice'], fold_one_pred))
        print('RMSE two: ', rmse_two)

        avg_rmse = np.mean([rmse_one, rmse_two])
        return avg_rmse
    
    else:
        # Instantiate sklearn's KFold function
        kf = KFold(n_splits=k, shuffle=True)
        
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            # Define training and testing set
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            
            # Fit model
            lr.fit(train[features], train["SalePrice"])
            
            # Predict on test set
            preds = lr.predict(test[features])
            
            # Calculate and append RMSE to list
            rmse = np.sqrt(mean_squared_error(test['SalePrice'], preds))
            rmse_values.append(rmse)
            
        avg_rmse = np.mean(rmse_values)

        return avg_rmse

In [6]:
# Test functions above
transform_df = transform_features(housing_data)
filtered_df = select_features(transform_df, 
                              coeff_threshold=0.4, uniq_threshold=10)
rmse = train_and_test(filtered_df, k=4)

rmse

28874.354458496986

In [7]:
def optimize(df, coeff_threshold=0.3, uniq_threshold=12, k=10):
    # Tweak pipeline parameters to find optimal solution
    transform_df = transform_features(housing_data)
    filtered_df = select_features(
        transform_df, coeff_threshold, uniq_threshold)
    rmse = train_and_test(filtered_df, k)
    print('c: ', coeff_threshold, 
          ' u: ', uniq_threshold, 
          ' k: ', k, 
          '\nrmse: ', round(rmse, 0))
    return rmse

In [8]:
optimize(housing_data)

c:  0.3  u:  12  k:  10 
rmse:  28703.0


28703.407322398736